get mass

In [25]:
%run -i 'get_mass.py'

In [12]:
import xml.etree.ElementTree as ET

In [13]:
tree = ET.parse('country_data.xml')
root = tree.getroot()

In [22]:
print(root[0][1].text)

2008


In [27]:
root = ET.fromstring('country_data')

ParseError: syntax error: line 1, column 0 (<string>)

In [29]:
from bs4 import BeautifulSoup

x="""<foo>
   <bar>
      <type foobar="1"/>
      <type foobar="2"/>
   </bar>
</foo>"""

y=BeautifulSoup(x)
print(y.foo.bar)

print(y.foo.bar.findAll("type"))
print(y.foo.bar.findAll("type")[0]["foobar"])

<bar>
<type foobar="1"></type>
<type foobar="2"></type>
</bar>
[<type foobar="1"></type>, <type foobar="2"></type>]
1


In [ ]:
from bs4 import BeautifulSoup


In [34]:
import requests
text = 'https://www.ebi.ac.uk/proteins/api/coordinates?offset=0&size=100&accession=P31749'
aa = requests.get(text)

In [156]:
y=BeautifulSoup(aa.content)
print(y.gnentries.gnentry.accession.text) #accession
print(y.gnentries.gnentry.sequence.text) #sequence
print(y.gnentries.gnentry.protein.recommendedname.fullname.text) #full-name
print(y.gnentries.gnentry.protein.findAll('alternativename')) # alternative names
print(y.gnentries.gnentry.findAll('gene')[1].text) #alternative gene names
print(y.gnentries.gnentry.findAll('gncoordinate')[0]['ensembl_gene_id']) # gene id
print(y.gnentries.gnentry.findAll('gncoordinate')[0]['ensembl_transcript_id']) # transc id
print(y.gnentries.gnentry.findAll('gncoordinate')[0]['ensembl_translation_id']) # translation id
print(y.gnentries.gnentry.findAll('gncoordinate')[0].genomiclocation['chromosome']) #chromosome
print(y.gnentries.gnentry.findAll('gncoordinate')[0].genomiclocation['reverse_strand']) # rev strand
print(y.gnentries.gnentry.findAll('gncoordinate')[0].genomiclocation['start']) #start
print(y.gnentries.gnentry.findAll('gncoordinate')[0].genomiclocation['end']) #ends
# feature type="modified residue"
# <feature type="modified residue">

P31749
MSDVAIVKEGWLHKRGEYIKTWRPRYFLLKNDGTFIGYKERPQDVDQREAPLNNFSVAQCQLMKTERPRPNTFIIRCLQWTTVIERTFHVETPEEREEWTTAIQTVADGLKKQEEEEMDFRSGSPSDNSGAEEMEVSLAKPKHRVTMNEFEYLKLLGKGTFGKVILVKEKATGRYYAMKILKKEVIVAKDEVAHTLTENRVLQNSRHPFLTALKYSFQTHDRLCFVMEYANGGELFFHLSRERVFSEDRARFYGAEIVSALDYLHSEKNVVYRDLKLENLMLDKDGHIKITDFGLCKEGIKDGATMKTFCGTPEYLAPEVLEDNDYGRAVDWWGLGVVMYEMMCGRLPFYNQDHEKLFELILMEEIRFPRTLGPEAKSLLSGLLKKDPKQRLGGGSEDAKEIMQHRFFAGIVWQHVYEKKLSPPFKPQVTSETDTRYFDEEFTAQMITITPPDQDDSMECVDSERRPHFPQFSYSASGTA
RAC-alpha serine/threonine-protein kinase
[<alternativename><fullname>RAC-PK-alpha</fullname></alternativename>, <alternativename><fullname>Proto-oncogene c-Akt</fullname></alternativename>, <alternativename><fullname>Protein kinase B alpha</fullname><shortname>PKB alpha</shortname></alternativename>, <alternativename><fullname>Protein kinase B</fullname><shortname>PKB</shortname></alternativename>]
PKB
ENSG00000142208
ENST00000554581
ENSP00000451828
14
true
104792643
104770344


In [154]:
data_modif_res = y.gnentries.gnentry.findAll('gncoordinate')[0].findAll('feature')
for entry in data_modif_res:
    if entry['type'] == 'modified residue': #only mod res
        if 'hospho' in entry.findAll('ns2:description')[0].text: #description is phosphosomething
            print(entry.findAll('ns2:description')[0].text) # what is mod. needs to order this
            print(entry.findAll('ns2:position')[0]['position']) # position of modification
            print(entry.findAll('ns2:evidence')[0].findAll('ns2:dbreference')[0]['id']) #needs a loop #pubmedid
            print(entry.findAll('ns2:evidence')[0].findAll('ns2:dbreference')[0]['type'])#needs a loop #pubmed

Phosphothreonine.
450
24275569
PubMed
Phosphotyrosine; by TNK2.
176
20333297
PubMed
Phosphoserine; by IKKE, MTOR and TBK1; alternate.
473
9736715
PubMed
Phosphoserine; alternate.
129
23186163
PubMed
Phosphothreonine; by IKKE, PDPK1 and TBK1.
308
15718470
PubMed
Phosphothreonine.
448
24275569
PubMed
Phosphoserine; alternate.
126
18669648
PubMed
Phosphoserine.
124
18669648
PubMed
Phosphotyrosine.
474
12149249
PubMed
